## Segmentação dos dados

As características normalmente são extraídas sobre pequenos segmentos de tamanho fixo dos dados, não no dado como um todo. Estes pequenos segmentos são chamados de janelas. A técnica de separar os dados em janelas, recebe o nome de janela deslizante (*sliding window*) e é uma forma de segmentação de dados. Uma boa prática, é definir um passo para essa janela de forma que haja uma sopreposição de dados, para que informações da lacuna de uma janela e outra não sejam perdidas. Na imagem a seguir podemos observar um sinal EOG. Abaixo dele a representação de janelas: *W1*, *W2*, *W3*... Repare que entre as janelas, há uma sobreposição de tamanho *T*. 

Na base de dados que iremos implementar, cada ensaio tem duração de 5 segundos. Se utilizadas janelas com tamanho de 250 ms, resultará em 20 janelas. Ao aplicar uma sobreposição de ~128ms, ficamos com 41 janelas de ~122ms.

Algumas características de EMG trabalham com o dado no domínio da frequência. Quando tais características são aplicadas, é necessário tranformar o dado para o domínio da frequência, utilizando o método de transformação de domínio `STFT`, do inglês *Short-time Fourier transform* (Transformada de Fourier de curto termo). O código divide o dado em segmentos, tanto no domínio do tempo quanto no domínio da frequência.

In [1]:
# Criando dados fictícios

import numpy as np

shape = (60, 4, 1000)
data = np.random.normal(loc=0, scale=10, size=shape).astype(np.float32)
data

array([[[-3.90393448e+00, -2.60749378e+01, -3.24774218e+00, ...,
          3.23517823e+00,  1.83841717e+00,  1.85505543e+01],
        [-3.10271597e+00,  1.09141512e+01,  9.76041126e+00, ...,
         -7.54746771e+00, -6.57333136e-01,  2.51514721e+01],
        [-7.00262547e+00, -1.31661568e+01, -1.79386640e+00, ...,
         -4.62527800e+00,  2.16235962e+01, -3.07737613e+00],
        [ 1.54577675e+01,  4.22393894e+00,  2.06021190e+00, ...,
         -1.90858040e+01,  7.59409952e+00, -6.39791906e-01]],

       [[ 1.33389962e+00, -1.37301931e+01, -3.43953633e+00, ...,
          7.52064371e+00,  5.72106028e+00,  5.33696461e+00],
        [ 5.54165077e+00,  7.98749971e+00,  1.11764002e+00, ...,
         -2.96858096e+00, -6.41626644e+00, -1.29623423e+01],
        [ 4.67923307e+00,  4.42317486e+00, -1.88987617e+01, ...,
          1.35019741e+01,  4.82803440e+00, -1.33640742e+00],
        [ 3.85384202e+00,  1.11449318e+01,  3.58403754e+00, ...,
         -1.84753168e+00, -1.00824518e+01, -2.21215

In [2]:
from scipy.signal import stft

step = 29
segment = 64
print('', data.shape)

n_win = int((data.shape[-1] - segment) / step) + 1
ids = np.arange(n_win) * step

# Janelas do dado no dominio do tempo
chunks_time = np.array([data[:,:,k:(k + segment)] for k in ids]).transpose(1, 2, 0, 3)

# Janelas do dado no domínio da frequência
_, _, chunks_freq = stft(data, fs=200, nperseg=64, noverlap=32)
chunks_freq = np.swapaxes(chunks_freq, 2, 3)

print('Formato (shape) dos dados depois da divisão de janelas')
print(f'Dominio do tempo: {chunks_time.shape} - (classes+ensaios, canais, janelas, linhas)')
print(f'Dominio da frequência:  {chunks_freq.shape} - (classes+ensaios, canais, janelas, linhas)')

 (60, 4, 1000)
Formato (shape) dos dados depois da divisão de janelas
Dominio do tempo: (60, 4, 33, 64) - (classes+ensaios, canais, janelas, linhas)
Dominio da frequência:  (60, 4, 33, 33) - (classes+ensaios, canais, janelas, linhas)


*Desafio*: Implementar um algoritmo para encontrar a mesma quantidade de janelas em ambos os domínios (tempo e frequência).

In [8]:
# Teste Desafio
segment = 64
nperseg_freq = 64
print('', data.shape)

overlap_time = 0  # Iniciando com sobreposição zero no domínio do tempo
overlap_freq = nperseg_freq // 2  # Sobreposição de 50% no domínio da frequência

while True:
    # Cálculo de n_win e criação de ids
    n_win = int((data.shape[2] - segment) / (overlap_time + 1)) + 1
    ids = np.arange(n_win) * (overlap_time + 1)

    # Janelas no domínio do tempo
    chunks_time = [data[:, :, k:(k + segment)] for k in ids]
    chunks_time = np.array(chunks_time).transpose(1, 2, 0, 3)

    # Janelas no domínio da frequência
    _, _, chunks_freq = stft(data, fs=200, nperseg=nperseg_freq, noverlap=overlap_freq)
    chunks_freq = np.swapaxes(chunks_freq, 2, 3)

    # Verificando se o número de janelas é o mesmo
    if chunks_time.shape[2] == chunks_freq.shape[2]:
        break  # Se o número de janelas for igual, saia do loop
    else:
        overlap_time += 1  # Incrementa a sobreposição no domínio do tempo para tentar igualar o número de janelas

print('Formato (shape) dos dados depois da divisão de janelas')
print(f'Dominio do tempo: {chunks_time.shape} - (classes+ensaios, canais, janelas, linhas)')
print(f'Dominio da frequência:  {chunks_freq.shape} - (classes+ensaios, canais, janelas, linhas)')

 (60, 4, 1000)
Formato (shape) dos dados depois da divisão de janelas
Dominio do tempo: (60, 4, 33, 65) - (classes+ensaios, canais, janelas, linhas)
Dominio da frequência:  (60, 4, 33, 33) - (classes+ensaios, canais, janelas, linhas)
